In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot
import numpy as np

In [ ]:
DATA_PATH = "/home/mathuis/Development/cyber_wolf/data"
HL_SIZE = 12

In [ ]:
def load_data(path: str):
    print("Loading data")
    return np.load(path, allow_pickle=True)


def build_model():
    print("Building model")
    
    input_layer = layers.Input(shape=(98,))
    encoded = layers.Dense(HL_SIZE, activation="relu")(input_layer)
    decoded = layers.Dense(98, activation="relu")(encoded)

    model = keras.Model(input_layer, decoded)    

    return model

def train_model(model, x_train, x_test):
    print("Compiling model")
    model.compile(optimizer="adam", loss="mse", metrics=["mse", "msle", "mae", "mape", "cosine_similarity"])
    print("Training model")
    return model.fit(x_train, x_train, epochs=6, batch_size=32, validation_data=(x_test, x_test))


def plot(history):
    print(history.keys())

    for key in history.keys():
        if not key.startswith("val_"):
            pyplot.plot()

            pyplot.title(key)
            pyplot.plot(history[key], label="train")
            pyplot.plot(history[f"val_{key}"], label="test")
            pyplot.xlabel("Epoch")
            pyplot.ylabel(key)
            pyplot.legend()

            pyplot.show()

def evaluate_model(model, x_test):
    results = model.evaluate(x_test, x_test)
    print(f"Test Loss: {results}")

In [ ]:
x_train = load_data(f"{DATA_PATH}/datasets/x_train.npy")
x_test = load_data(f"{DATA_PATH}/datasets/x_test.npy")
    

In [ ]:
model = build_model()
model.summary()

In [ ]:
res = train_model(model, x_train, x_test)

In [ ]:
# print(res.history.keys())
plot(res.history)

In [ ]:
evaluate_model(model, x_test[:1])

In [ ]:
pred = model.predict(x_test)
print(x_test[0])
print(pred[0])

In [ ]:
# model.save(f"{DATA_PATH}/models/1h-12n-6e-b32-mse_loss-notnorm-model.h5")